In [ ]:
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2

In [ ]:
# from gnn_tracking.postprocessing.clusterscanner import ClusterHyperParamScanner
import optuna
import numpy as np
from sklearn.cluster import DBSCAN

from sklearn.datasets import make_blobs

from sklearn import metrics

In [ ]:
# def gen_sample():
#     # n_clusters = np.random.randint(2, 4)
#     # centers = np.random.uniform(0, 5, size=(n_clusters, 2))
#     # X, labels_true = make_blobs(
#     #     n_samples=750, centers=centers, cluster_std=0.4, random_state=0
#     # )
#     centers = [[1, 1], [-1, -1], [1, -1]]
#     X, labels_true = make_blobs(
#         n_samples=750, centers=centers, cluster_std=0.4
#     )
#     return X, labels_true

In [ ]:
from pathlib import Path

datadir = Path("~/data/gnn_tracking/clust").expanduser()
names = [
    "0_0",
    "1_5",
    "2_8",
    "3_9",
    "4_15",
    "5_16",
    "6_20",
    "7_25",
    "8_30",
    "9_31",
]
graphs = []
truths = []
for name in names:
    graphs.append(np.load(datadir / f"{name}.npy"))
    truths.append(np.load(datadir / f"t_{name}.npy"))

In [ ]:
truths[0].shape

In [ ]:
for i in range(10):
    plt.scatter(graphs[i][:, 0], graphs[i][:, 1], c=truths[i])
    plt.show()

In [ ]:
len(np.unique(truths[0]))

In [ ]:
len(truths[0])

In [ ]:
from abc import ABC, abstractmethod
from typing import Callable
import pandas as pd


def expensive_metric(*, truth, predicted) -> float:
    c_id = pd.DataFrame({"c": predicted, "id": truth})
    clusters = c_id.groupby("c")
    # majority_pid = clusters['id'].apply(lambda x: x.mode()[0])
    majority_counts = clusters["id"].apply(lambda x: sum(x == x.mode()[0]))
    majority_fraction = clusters["id"].apply(lambda x: sum(x == x.mode()[0]) / len(x))
    h_id = pd.DataFrame({"hits": np.ones(len(predicted)), "id": truth})
    particles = h_id.groupby("id")
    nhits = particles["hits"].apply(lambda x: len(x)).to_dict()
    majority_hits = clusters["id"].apply(lambda x: x.mode().map(nhits)[0])
    # perfect_match = ((majority_hits==majority_counts) &
    #                  (majority_fraction > 0.99))
    double_majority = ((majority_counts / majority_hits).fillna(0) > 0.5) & (
        majority_fraction > 0.5
    )
    return sum(double_majority) / len(predicted)

In [ ]:
from gnn_tracking.postprocessing.clusterscanner import ClusterHyperParamScanner
from gnn_tracking.utils.earlystopping import RelEarlyStopper

es = RelEarlyStopper(wait=30, grace=50, change_threshold=0.01, direction="max")


def dbscan(graph, eps, min_samples):
    return DBSCAN(eps=eps, min_samples=min_samples).fit_predict(graph)


def cheap_metric(labels, truth):
    return metrics.v_measure_score(truth, labels)


chps = ClusterHyperParamScanner(
    dbscan,
    graphs,
    truths,
    expensive_metric,
    cheap_metric=metrics.v_measure_score,
    early_stopping=es,
)

In [ ]:
study = chps.scan(n_trials=10)

In [ ]:
from optuna.visualization import plot_contour

In [ ]:
study.best_params

In [ ]:
plot_contour(study)

In [ ]:
from optuna.visualization import plot_slice

plot_slice(study)

In [ ]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(study)